In [1]:
import pandas as pd
import numpy as np
from datetime import date
from scipy.sparse import coo_matrix

data_df = pd.read_csv('./ratings.csv', sep=',', names=["UserID", "MovieID", "Rating", "Timestamp"],dtype={'Rating':'float64'})
 #dtype={'UserID':'int64','MovieID':'int64','Rating':'float64','Timestamp':'float'}
#dataframe object with movie information
#movie_data_df = pd.read_csv('./movies_metadata.csv', sep=',', names=["movieId", "title", "genres"])

#data_df['Timestamp'] = pd.to_datetime(data_df['Timestamp'])
data_df['Timestamp'] = pd.to_datetime(data_df['Timestamp'], unit='s')
cutoff_date = "2017-02-01" # this was the earliest i could do without a memory error
data_df = data_df[data_df['Timestamp'] >= cutoff_date]
                    
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())
num_ratings = len(data_df['Rating'])
print(num_ratings)

print(train_df.info())
# train_mat = train_df[['Rating', 'UserID']].copy()
# test_mat = test_df[['Rating', 'UserID']].copy()
train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

1090556
<class 'pandas.core.frame.DataFrame'>
Int64Index: 763001 entries, 151 to 26023521
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   UserID     763001 non-null  int64         
 1   MovieID    763001 non-null  int64         
 2   Rating     763001 non-null  float64       
 3   Timestamp  763001 non-null  datetime64[ns]
 4   movieID    763001 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 34.9 MB
None


In [2]:
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

print(train_mat)

[[1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]]


user-user Collaborative Filtering with implicit feedback SWITCH TO ITEM-ITEM

In [3]:
#implicit user user collab filtering

user_train_like = []
for u in range(num_user):
    user_train_like.append(np.where(train_mat[u, :] > 0)[0])

numer = np.matmul(train_mat, train_mat.T)
denom = np.sum(train_mat ** 2, axis=1, keepdims=True) ** 0.5
Cosine = numer / np.matmul(denom, denom.T)

recommendation = []
for u in range(num_user):
    similarities = Cosine[u, :]
    similarities[u] = -1
    N_idx = np.argpartition(similarities, -10)[-10:]
    N_sim = similarities[N_idx]
    scores = np.sum(N_sim.reshape((-1, 1)) * train_mat[N_idx, :], axis=0) / np.sum(N_sim)
    
    train_like = user_train_like[u]
    scores[train_like] = -9999
    top50_iid = np.argpartition(scores, -50)[-50:]
    top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
    recommendation.append(top50_iid)
recommendation = np.array(recommendation)

<ipython-input-3-596c2195aa96>:9: RuntimeWarning: invalid value encountered in true_divide
  Cosine = numer / np.matmul(denom, denom.T)


In [4]:
print(recommendation)

[[28717  9569  9560 ...  9592  9591  9578]
 [28717  9547  9538 ...  9570  9569  9556]
 [28717  9547  9538 ...  9570  9569  9556]
 ...
 [28717  9547  9538 ...  9570  9569  9556]
 [28717  9558  9549 ...  9581  9580  9567]
 [14358  9638  9629 ...  9660  9659  9646]]


In [5]:
# Calculate recall@k, precision@k with k=5, 20, 50 and print out the average over all users for these 6 metrics.
# Your Code Here...

user_test_like = []
for u in range(num_user):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])
    
recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

for u in range(num_user):
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))

recall@5	[0.000011],	||	 recall@20	[0.000107],	||	 recall@50	[0.000206]
precision@5	[0.000160],	||	 precision@20	[0.000266],	||	 precision@50	[0.000288]
